## SocialAL Model
# Fit model to data - one subject with variable prior probability
KLS 2.03.21  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python Version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

### Define functions

In [3]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    
    # initialize variables
    #prob = [0.5, 0.5, 0.5] allowed to vary for best-fit
    #ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    ev = [update_value(prob[0]), update_value(prob[1]), update_value(prob[2])]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        print('Trial: ' + str(trial +1))
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)
            ev[trustee] = update_value(prob[trustee])
            #print('choice prob: ', cprob)
        #print("Choice: ", choice)
        #print('Prob for each trustee: ', prob)
        #print('EV for Trustee:', ev[trustee])
        #print('EV for all:', ev)
        #print('Probs: ', probs) 
        print('LLH: ', totalLLH)
        #print('Trustee Prob: ', prob[trustee])
        #print('Trustee Expected values: ', ev[trustee])
        
    return totalLLH

### Load and clean data

In [4]:
#os.listdir('../data/modeling') 

In [5]:
#dt = pd.read_csv('../data/modeling/sub-1004.csv')
dt = pd.read_csv('../data/modeling/sub-1005.csv')
#dt = pd.read_csv('../data/modeling/sub-1008.csv') #- try getting this sub to work
#dt = pd.read_csv('../data/modeling/sub-2013.csv') #- try getting this sub to work
#dt = pd.read_csv('../data/modeling/sub-2032.csv') #- try getting this sub to work
# recode trial type into numbers for model
def stims(trial_type):
    if trial_type == "Trustworthy":
        return 0
    elif trial_type == "Neutral":
        return 1
    elif trial_type == "Untrustworthy":
        return 2
dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
# rename response_key to choice
def choices(response_key):
    if response_key == 'None':
        return 0 
    else:
        return response_key  
dt['Choice'] = dt['response_key'].apply(choices)
dt['Choice'] = pd.to_numeric(dt['Choice'])
# calculte the trustee response
def resp(trial_earnings):
    if trial_earnings >= 12:
        return 1
    else:
        return 0
dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]

In [6]:
# Input Parameters

array = [1.61830799e-01, 4.86765198e-01, 4.70079839e-01, 4.91178652e-01, 1.87341150e-01, 1.56635508e-12]

get_likelihood_action(array, data)

Trial: 1
LLH:  0.5046431063277187
Trial: 2
LLH:  1.8911339438749883
Trial: 3
LLH:  2.2664196920040114
Trial: 4
LLH:  2.6417054401330344
Trial: 5
LLH:  4.328215273197873
Trial: 6
LLH:  6.0147251062641995
Trial: 7
LLH:  6.290995422332452
Trial: 8
LLH:  7.69011686035336
Trial: 9
LLH:  7.9663871764216125
Trial: 10
LLH:  9.48072338799718
Trial: 11
LLH:  11.16723322106427
Trial: 12
LLH:  12.683924182722299
Trial: 13
LLH:  12.960194498790377
Trial: 14
LLH:  15.017814685597315
Trial: 15
LLH:  15.899861395762882
Trial: 16
LLH:  16.803887181928957
Trial: 17
LLH:  18.66406062313092
Trial: 18
LLH:  20.329500567093206
Trial: 19
LLH:  21.65363526734948
Trial: 20
LLH:  23.92197370705096
Trial: 21
LLH:  24.466378252403565
Trial: 22
LLH:  24.742648568471644
Trial: 23
LLH:  26.14223628981519
Trial: 24
LLH:  27.579074821500072
Trial: 25
LLH:  29.09341100630865
Trial: 26
LLH:  29.5039309946139
Trial: 27
LLH:  31.00592289652962
Trial: 28
LLH:  32.33704624273145
Trial: 29
LLH:  34.02355607579893
Trial: 30
L

41.6106468320843